<div class="alert alert-block alert-info"> <b>NOTE</b> Please select the kernel <code>Python [conda env: deepcell]</code> for this notebook. </div>

# 1. Data Preparation
Resave data as a set of tiff files in order to match Cell Tracking Challenge conventions which are expected by Baxter

In [5]:
import os
import sys

import numpy as np
from tifffile import imwrite

from deepcell.applications import NuclearSegmentation
from deepcell_tracking.isbi_utils import trk_to_isbi
from deepcell_tracking.trk_io import load_trks

sys.path.append('..')
import utils

In [13]:
data_dir = 'data-ctc'
raw_dir = os.path.join(data_dir, 'raw')
seg_gt_dir = os.path.join(raw_dir, 'Analysis/Segmentation_GT')
seg_dc_dir = os.path.join(raw_dir, 'Analysis/Segmentation_DeepCell')
gt_dir = os.path.join(data_dir, 'GT')

for d in [raw_dir, seg_gt_dir, seg_dc_dir, gt_dir]:
    if not os.path.exists(d):
        os.makedirs(d)

Load the test split of the tracking data

In [10]:
files = ['../../data/Fluo-N2DL-Hela-test/Fluo-N2DL-HeLa-test-01.trk', '../../data/Fluo-N2DL-Hela-test/Fluo-N2DL-HeLa-test-02.trk']
X, y, lineages = [], [], []
for f in files:
    data = load_trks(f)
    X.append(data['X'])
    y.append(data['y'])
    lineages.append(data['lineages'][0])

data = {
    'X': np.stack(X, axis=0),
    'y': np.stack(y, axis=0),
    'lineages': lineages
}

Load the DeepCell nuclear segmentation model to test the algorithm on predicted instead of ground truth segmentations

In [11]:
app_seg = NuclearSegmentation.from_version(version='1.1')

INFO:root:Checking for cached data
INFO:root:Checking NuclearSegmentation-8.tar.gz against provided file_hash...
INFO:root:NuclearSegmentation-8.tar.gz with hash 507be21f0e34e59adae689f58cc03ccb already available.
INFO:root:Extracting /Users/morganschwartz/.deepcell/models/NuclearSegmentation-8.tar.gz
INFO:root:Successfully extracted /Users/morganschwartz/.deepcell/models/NuclearSegmentation-8.tar.gz into /Users/morganschwartz/.deepcell/models


Metal device set to: Apple M1 Max

systemMemory: 64.00 GB
maxCacheSize: 24.00 GB



2024-07-08 14:57:58.560784: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-07-08 14:57:58.560911: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Convert each batch of the test split to the standard ISBI format which is compatible with most of the models that we will test.

In [14]:
for batch_no in range(len(data['lineages'])):
    # Build subdirectories for data
    raw_subdir = os.path.join(raw_dir, '{:03}'.format(batch_no + 1))
    gt_subdir = os.path.join(gt_dir, '{:03}'.format(batch_no + 1))
    seg_gt_subdir = os.path.join(seg_gt_dir, '{:03}'.format(batch_no + 1))
    seg_dc_subdir = os.path.join(seg_dc_dir, '{:03}'.format(batch_no + 1))

    # Create directories if needed
    for d in (raw_subdir, gt_subdir, seg_gt_subdir, seg_dc_subdir):
        if not os.path.exists(d):
            os.makedirs(d)

    # Pull out relevant data for this batch
    X = data['X'][batch_no]
    y = data['y'][batch_no]
    lineage = data['lineages'][batch_no]

    # Correct discontiguous tracks, which are not allowed by CTC
    y, lineage = utils.convert_to_contiguous(y, lineage)

    # Determine position of zero padding for removal
    slc = utils.find_zero_padding(X)
    X = X[slc]
    y = y[slc]

    # Determine which frames are zero padding
    frames = np.sum(y, axis=(1,2)) # True if image not blank
    good_frames = np.where(frames)[0]
    X = X[:len(good_frames)]
    y = y[:len(good_frames)]

    # Need to translate lineages and adjust images to match restrictive ISBI format
    # Prepare output txt
    text_file = os.path.join(gt_subdir, 'man_track.txt')
    df = trk_to_isbi(lineage)
    df.to_csv(text_file, sep=' ', header=False, index=False)

    # Save each frame of the movie as an individual tif
    channel = 0 # These images should only have one channel
    for i in range(X.shape[0]):
        name_raw = os.path.join(raw_subdir, 't{:03}.tif'.format(i))
        name_seg_gt = os.path.join(seg_gt_subdir, 't{:03}.tif'.format(i))
        name_seg_dc = os.path.join(seg_dc_subdir, 't{:03}.tif'.format(i))
        name_gt = os.path.join(gt_subdir, 't{:03}.tif'.format(i))

        # Generate deepcell prediction
        pred = app_seg.predict(X[i:i+1, ..., channel:channel+1])

        imwrite(name_raw, X[i, ..., channel])
        imwrite(name_seg_gt, y[i, ..., channel].astype('uint16'))
        imwrite(name_seg_dc, pred.astype('uint16'))
        imwrite(name_gt, y[i, ..., channel].astype('uint16'))

INFO:root:Converting image dtype to float


INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype t

# 2. Tracking

The steps for tracking will be performed in Matlab and the Baxter Algorithm GUI. You will need access to Matlab 2019b. The instructions below outline the general workflow for generating benchmarking results using the Baxter algorithm. However additional support for using the software is available in the user guide located in the KTH-SE folder (`KTH-SE/UserGuide/UserGuide.pdf`).

## Part 1 - Setup
1. Open the KTH-SE folder in Matlab
2. Open `BaxterAlgorithms.m` in the Matlab editor and then hit Run. This should open a GUI.
3. Within the GUI, select `Files > Open experiment > Browse...`. Navigate to the `data/raw` folder located within this Baxter folder.
4. Ensure that all files are selected in the list on the right hand side of the GUI.
## Part 2 - GT Segmentations
5. Select `Settings > Load Settings (Browse for files)`. Navigate to `Settings-SegmentationGT.csv` within the Baxter folder. In the following prompts, ensure that segmentations are applied to all images.
6. Select `Automated > Track`. In the pop-up window, change the `Save Version` to "GT" and press `Start`. 
7. Select `File > Export tracks to CTC format > RES tracks` and follow the prompts in order to export the tracking results to the standard CTC format. Make sure to select the "GT" version for export.
## Part 3 - DeepCell Segmentations
8. Select `Settings > Load Settings (Browse for files)`. Navigate to `Settings-SegmentationDeepCell.csv` within the Baxter folder. In the following prompts, ensure that segmentations are applied to all images.
6. Select `Automated > Track`. In the pop-up window, change the `Save Version` to "DeepCell" and press `Start`. 
7. Select `File > Export tracks to CTC format > RES tracks` and follow the prompts in order to export the tracking results to the standard CTC format. Make sure to select the "DeepCell" version for export.


# 3. Evaluation

In [15]:
import os
import re
import shutil
import subprocess

import pandas as pd

from deepcell_tracking.metrics import TrackingMetrics

In [16]:
data_dir = 'data-ctc'
gt_dir = os.path.join(data_dir, 'GT')
export_gt_dir = os.path.join(data_dir, 'raw/Analysis/CellDataGT/RES/')
export_dc_dir = os.path.join(data_dir, 'raw/Analysis/CellDataDeepCell/RES/')
ctc_gt = os.path.join(data_dir, 'SEG_GT')
ctc_dc = os.path.join(data_dir, 'SEG_PRED')

data_ids = os.listdir(gt_dir)

node_match_threshold = 0.6

ctc_software = '../CTC_Evaluation_Software'
operating_system = 'Mac' # or 'Mac' or 'Win'
num_digits = '3'

In order to utilize the CTC evaluation functions, the data must be reorganized to match CTC conventions.

In [17]:
# Copy the GT data over to appropriate folders
for target in [ctc_gt, ctc_dc]:
    for d in os.listdir(gt_dir):
        seg_dir = os.path.join(target, f'{d}_GT/SEG')
        tra_dir = os.path.join(target, f'{d}_GT/TRA')
        if not os.path.exists(seg_dir): os.makedirs(seg_dir)
        if not os.path.exists(tra_dir): os.makedirs(tra_dir)

        for f in os.listdir(os.path.join(gt_dir, d)):
            start_path = os.path.join(gt_dir, d, f)

            # Only copy the txt file with tracks to TRA dir
            if f.endswith('txt'):
                end_path = os.path.join(tra_dir, f)
                shutil.copy(start_path, end_path)
            else:
                # Copy to seg_dir and modify name
                end_path = os.path.join(seg_dir, f'man_seg{f[1:]}')
                shutil.copy(start_path, end_path)

                # Copy to tra dir and modify name
                end_path = os.path.join(tra_dir, f'man_track{f[1:]}')
                shutil.copy(start_path, end_path)

# Copy over results
for target, start_dirs in zip([ctc_gt, ctc_dc], [export_gt_dir, export_dc_dir]):
    for d in os.listdir(start_dirs):
        start_dir = os.path.join(start_dirs, d)
        end_dir = os.path.join(target, d)
        shutil.copytree(start_dir, end_dir)

In [18]:
benchmarks = []

for results_dir, s in zip([ctc_gt, ctc_dc], ['GT', 'Deepcell']):
    for data_id in data_ids:
        print(data_id)
        results = {
            'model': f'Baxter - {s}',
            'data_id': data_id
        }
        gt_dir = os.path.join(results_dir, f'{data_id}_GT/TRA')
        res_dir = os.path.join(results_dir, f'{data_id}_RES')

        # Deepcell benchmarking
        m = TrackingMetrics.from_isbi_dirs(gt_dir, res_dir, threshold=node_match_threshold)
        results.update(m.stats)

        # CTC metrics
        for metric, path in [('DET', 'DETMeasure'), ('SEG', 'SEGMeasure'), ('TRA', 'TRAMeasure')]:
            p = subprocess.run([os.path.join(ctc_software, operating_system, path), results_dir, data_id, num_digits],
                               stdout=subprocess.PIPE)
            outstring = p.stdout

            try:
                val = float(outstring.decode('utf-8').split()[-1])
                results[metric] = val
            except:
                print('Benchmarking failure', path, results_dir, data_id)
                print(outstring.decode('utf-8'))

        benchmarks.append(results)

df = pd.DataFrame(benchmarks)
df.to_csv('ctc-benchmarks.csv')

001
missed node 5_1 division completely
missed node 29_14 division completely
missed node 32_77 division completely
missed node 38_17 division completely
missed node 42_14 division completely
missed node 46_20 division completely
missed node 49_10 division completely
missed node 56_37 division completely
missed node 72_62 division completely
missed node 79_73 division completely
missed node 80_78 division completely
missed node 94_53 division completely
missed node 97_34 division completely
missed node 100_43 division completely
missed node 107_46 division completely
missed node 417_51 division completely
missed node 119_50 division completely
425_47 out degree = 2, daughters mismatch, gt and res degree equal.
missed node 122_61 division completely
missed node 126_76 division completely
missed node 145_81 division completely
missed node 153_77 division completely
missed node 402_88 division completely
missed node 156_82 division completely
missed node 163_89 division completely
missed 

In [19]:
df

,model,data_id,correct_division,mismatch_division,false_positive_division,false_negative_division,total_divisions,aa_tp,aa_total,te_tp,te_total,DET,SEG,TRA
0,Baxter - GT,001,122,1,22,38,161,13256,13508,13682,13944,0.995432,0.995167,0.994721
1,Baxter - GT,002,94,2,28,37,133,17140,17465,17579,17949,0.993398,0.991611,0.992112
2,Baxter - Deepcell,001,119,0,21,42,161,13351,13508,13784,13944,0.993244,0.894491,0.992673
3,Baxter - Deepcell,002,93,3,25,37,133,17244,17465,17706,17949,0.990228,0.892680,0.988604
